In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

import time
import operator
import re

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = 10, 8
plt.rcParams['axes.unicode_minus'] = False

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

### 데이터를 읽어온다.

In [2]:
df = pd.read_csv('data10/review_data.csv')
df.head()

,score,review,y
0,5,친절하시고 깔끔하고 좋았습니다,1
1,5,조용하고 고기도 굿,1
2,4,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",1
3,4,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,1
4,5,고기도 맛있고 서비스는 더 최고입니다~,1


### 한글 외의 모든 글자를 제거한다.

In [3]:
def text_cleaning(text):
    # 한글 정규식
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    
    # 한글 정규식에 해당하지 않는 것은 길이가 0인 문자열로 변환한다.
    result = hangul.sub('', text)
    return result

In [4]:
# 리뷰 내용을 정제한다.
df['ko_text'] = df['review'].apply(lambda x : text_cleaning(x))
del df['review']

df

,score,y,ko_text
0,5,1,친절하시고 깔끔하고 좋았습니다
1,5,1,조용하고 고기도 굿
2,4,1,갈비탕과 냉면 육회비빔밥이 맛있습니다
3,4,1,대체적으로 만족하나와인의 구성이 살짝 아쉬움
4,5,1,고기도 맛있고 서비스는 더 최고입니다
...,...,...,...
540,3,0,추웟어요 고기 외에는 별로에요
541,1,0,고기질과 육전은 좋다다만 한우손님 돼지고기 손님을 차별한다돼지손님은 주차불가네이버예...
542,5,1,직접 구워주시고 진짜맛있음 반찬도 맛있음 직원분이 친절하게 잘해주시네요
543,4,1,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ


### 형태소 분석

In [5]:
from konlpy.tag import Okt

# konlpy 라이브러리 텍스트를 데이터에서 형태소를 추출한다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    # print(pos)
    
    result = []
    
    for a1 in pos:
        result.append(f'{a1[0]}/{a1[1]}')
        
    return result

get_pos(df['ko_text'][0])

['친절하시고/Adjective', '깔끔하고/Adjective', '좋았습니다/Adjective']

### 분류 모델의 학습 데이터로 변환하기

In [6]:
# 단어 카운터 벡터를 만들어줌
from sklearn.feature_extraction.text import CountVectorizer

# 형태소를 벡터 형태의 학습 데이터셋으로 변환한다.
index_vectorizer = CountVectorizer(tokenizer = lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())
X

<545x3030 sparse matrix of type '<class 'numpy.int64'>'
	with 9692 stored elements in Compressed Sparse Row format>

In [7]:
# 품사와 단어 개수를 확인
# index_vectorizer.vocabulary_

In [8]:
num = 10
print(df['ko_text'][num])
print(X[num])

맛있어요 양이 적지만 한우니까요 ㅋㅋ한우초밥도 맛있었습니다
  (0, 721)	1
  (0, 2082)	1
  (0, 2)	1
  (0, 2884)	2
  (0, 1017)	1
  (0, 1810)	1
  (0, 2296)	1
  (0, 932)	1
  (0, 604)	1
  (0, 2019)	1
  (0, 2610)	1
  (0, 1029)	1


### TF-IDF로 변환한다.

In [9]:
# 위에서 만든 형태소 벡터를 학습 데이터 벡터로 생성한다.
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)
print(X[0])

  (0, 2647)	0.5548708693511647
  (0, 2403)	0.48955631270748484
  (0, 428)	0.6726462183300624


### 학습 데이터를 생성한다.

In [10]:
from sklearn.model_selection import train_test_split

# 결과
y = df['y']

# 학습용과 검증용으로 나눈다.
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
print(x_train.shape)
print(x_test.shape)

(381, 3030)
(164, 3030)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [12]:
# 로지스틱 회귀 모델을 생성한다.
lr = LogisticRegression()

# 학습
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
# 테스트용 데이터를 통해 예측한다.
y_pred = lr.predict(x_test)

In [14]:
# 평가한다.
r1 = accuracy_score(y_test, y_pred)
r2 = precision_score(y_test, y_pred)
r3 = recall_score(y_test, y_pred)
r4 = f1_score(y_test, y_pred)
r5 = roc_auc_score(y_test, y_pred)

In [15]:
print(f'accuracy : {r1}')
print(f'precision : {r2}')
print(f'recall : {r3}')
print(f'f1 : {r4}')
print(f'roc_auc : {r5}')


accuracy : 0.8902439024390244
precision : 0.8902439024390244
recall : 1.0
f1 : 0.9419354838709678
roc_auc : 0.5


### 중요 피처 형태소

In [16]:
# 인덱스를 설정하여 리스트형태로 만든다.
list(enumerate(lr.coef_[0]))

# 회귀 모델의 계수를 높은 순서대로 정렬한다.
a1 = list(enumerate(lr.coef_[0]))

# sorted 함수를 쓰면 첫번째 것을 기준으로 정렬하기 때문에
# index, 계수 형태를 계수, index 형태로 바꿔준다.
a2 = list(((value, index) for index, value in a1))

# 정렬한다.
coef_pos_index = sorted(a2, reverse=True)
# coef_pos_index

In [19]:
# 단어 번호 : 단어 형태의 딕셔너리를 생성한다.
text_data_dict = {}

for text, text_id in index_vectorizer.vocabulary_.items():
    text_data_dict[text_id] = text

In [20]:
# 긍정단위 상위 200개
coef_pos_index[:20]

[(0.7409205533276514, 1017),
 (0.6166561056628442, 1001),
 (0.4919714916108256, 1093),
 (0.43301124016576736, 721),
 (0.4284004241532582, 1030),
 (0.42468039084310166, 999),
 (0.4245107191046603, 2246),
 (0.3576442448899036, 873),
 (0.3555472071515358, 2247),
 (0.353527295546285, 1029),
 (0.34774651653182537, 2330),
 (0.34221372731051863, 2613),
 (0.338436645184621, 2376),
 (0.3378695631667706, 2403),
 (0.333499661571251, 2404),
 (0.3133079946166147, 1388),
 (0.3010529706680342, 2502),
 (0.29988312887881974, 2647),
 (0.28674014996370867, 1920),
 (0.2754230223630745, 1007)]

In [21]:
# 부정단위 상위 20개
coef_pos_index[-20:]

[(-0.4382553820347116, 2082),
 (-0.4775682749788326, 2198),
 (-0.5049940960456706, 2289),
 (-0.5166780316625493, 1720),
 (-0.5374363970783291, 2371),
 (-0.5796998339354771, 1608),
 (-0.5796998339354771, 18),
 (-0.5804747994290431, 338),
 (-0.5831336643390982, 1555),
 (-0.5851684394472948, 24),
 (-0.6600404847918911, 16),
 (-0.667645532017591, 2365),
 (-0.6770251419048888, 1508),
 (-0.7075829725236282, 2427),
 (-0.7130337368827411, 901),
 (-0.7766913643487544, 589),
 (-0.8198193672569826, 399),
 (-0.9462753743779084, 2070),
 (-1.18033042678668, 1309),
 (-1.3556489705890737, 2069)]

In [22]:
# coef_pos_index 안에 있는 값들 중 단어 번호를 실제 단어로 변환한다.
coef_pos_text = []

for value, index in coef_pos_index:
    # index에 해당하는 단어를 추출한다.
    text = text_data_dict[index]
    
    # 계수와 단어의 조합으로 만들어 담아준다.
    coef_pos_text.append((value, text))
    
# 상위 20개
for text in coef_pos_text[:20] :
    print(text)
    
# 하위 20개
print('------------------')
for text in coef_pos_text[-20:] :
    print(text)

(0.7409205533276514, '맛있어요/Adjective')
(0.6166561056628442, '맛있고/Adjective')
(0.4919714916108256, '먹었습니다/Verb')
(0.43301124016576736, '도/Josa')
(0.4284004241532582, '맛있었어요/Adjective')
(0.42468039084310166, '맛있게/Adjective')
(0.4245107191046603, '잘/Verb')
(0.3576442448899036, '또/Noun')
(0.3555472071515358, '잘/VerbPrefix')
(0.353527295546285, '맛있었습니다/Adjective')
(0.34774651653182537, '정말/Noun')
(0.34221372731051863, '최고/Noun')
(0.338436645184621, '좋고/Adjective')
(0.3378695631667706, '좋았습니다/Adjective')
(0.333499661571251, '좋았어요/Adjective')
(0.3133079946166147, '분위기/Noun')
(0.3010529706680342, '진짜/Noun')
(0.29988312887881974, '친절하시고/Adjective')
(0.28674014996370867, '역시/Noun')
(0.2754230223630745, '맛있는/Adjective')
------------------
(-0.4382553820347116, '이/Josa')
(-0.4775682749788326, '있는데/Adjective')
(-0.5049940960456706, '적어요/Verb')
(-0.5166780316625493, '안/Noun')
(-0.5374363970783291, '종업원/Noun')
(-0.5796998339354771, '시끄러워요/Adjective')
(-0.5796998339354771, 'ㅠㅠㅠㅠ/KoreanParticle')
(-0.5